# Variables & Memory References

Generally, first few bits in the memory for an object is used for storing the type of theo object.

In [1]:
 my_var_1 = 10

In [2]:
id(my_var_1) #Memory location in base 10 number

140725573886640

Unlike other programming languages, Python doesn't store the value inside the varaible. In the above example the value **10** is not actually equal to **my_var_1**. The variable **my_var_1** is actually a **pointer** to the **integer 10** which is stored in the memory, and hence the my_var_1 variable contains the location to the integer 10.

In plain english:  
my_var_1 **reference** the object at _140719859724832_ memory location

In [3]:
my_var_2 = 'hello'

In [4]:
id(my_var_2)

2185270781552

In plain english:  
my_var_2 **reference** the object at _3048064126256_ memory location

# Reference Counting

Reference counting is having the count of the references in the program for all the objects.
Like in the above case the integer 10, string 'hello' is being *referred* to my_var_1 and my_var_2 respectively, so both the object 10 and *hello* has a reference count of 1

### Side Note: Memory Leak

Memory leak is a occured when the part of memory(heap) is not usable/needed. In other words, the program incorrectly manages memory allocations in a way that memory which is no longer needed is not released.  
A memory leak may also happen when an object is stored in memory but cannot be accessed by the running code.

The Python Memory Manager, keeps looking for the reference counts of the objects, and when there is no reference to a particular object it **removes(*forgets*)** the location of the object in the memory in the program.

### Reference Counting Method 1  
Not Advised

In [5]:
import sys

In [6]:
a = [1, 2, 3, 1, 3, '4', 's']

In [7]:
id(a)

2185260479752

In [8]:
print(id(a[0]), id(a[3]), id(1))
id(a[0]) == id(a[3]) == id(1) #Integer interning

140725573886352 140725573886352 140725573886352


True

In [9]:
sys.getrefcount(a) #all good, but remember to subtract 1, as getrefcount is adding itself too

2

Even though the list is being refered only by **a**, but we get the no. of references as **2**, this is because the sys.getrefcount also referes to the list and increase the reference count by 1.

### Reference Counting Method 2  
Advisable to use dureing **Debugging**

In [10]:
import ctypes
def get_ref_count(address: int):
    return ctypes.c_long.from_address(address).value

In [11]:
ab = [1, "TSAI is fun", {3,4,5,'eva'}]

In [12]:
get_ref_count(id(ab))

1

Using cytpes, we can get the exact number of reference for the object, since ctypes doesn't create a reference like sys.getrefcount

In [13]:
get_ref_count(id(a))

1

In [14]:
w = [1, 2, 3, 'red ball', 'breaking bad']
id_w = id(w) #The id of memory where the object(list) is stored(pointed by w)
                #is being pointed by id_w
v = w
print(get_ref_count(id_w)) #No. of references to list object
w = None
print(get_ref_count(id_w)) #No. of references to list object
v = None
print(get_ref_count(id_w))
id_w = id(w)
print(get_ref_count(id_w)) #No. of references to None object
v = None

2
1
1
26612


In [15]:
another_w = [2335, 'Boy', 12]
another_id_w = id(another_w)
another_v = another_w
print(get_ref_count(another_id_w)) #The List object has two ref v and w
another_w = None
another_v = None

2


In [16]:
print(get_ref_count(another_id_w)) #The list object is now no more referenced

0


# Garbage Collection

Python Memory Manager: It keeps a count of references and when the references for the object goes to ZERO, the python memory manager deletes the object(forgets it from the program memory) and reclaims the memory to use again.  
But sometimes that doesn't work as expected. And in particular, we have to look at a situation called Circular References.

Circular References: A circular reference is a series of references where the last object references the first, resulting in a closed loop.

##### Case 1
my_var = object_A #(Instance of Class A)
object_A.var1 = object_B #(Instance of Class B)

Now del my_var  
References to object_A is now **zero**, PMM removes object_A and also object_A.var1  
Now, reference to object_B became **zero**, PMM removes object_B

All the memory is cleared where the unused objects are removed.

##### Case 2
my_var = object_A #(Instance of Class A)
object_A.var1 = object_B #(Instance of Class B)
object_B.var2 = object_A #(object_A now has 2 references)

Now del my_var  
References to object_A is now **one**, PMM **doesn't** remove object_A and also object_A.var1
Now, reference to object_B is also **one**, so PMM **doesn't** removes object_B

But the program doesn't use object_A and object_B, since the main reference my_var is no more avialbe and both the object_A and object_B will remain in memory. This is Cyclic Reference and a memory leak!  

**Garbage Collector** now comes into picture, GC always runs after certain amount of time (60 seconds) and checks for all cyclic references and removes them.

In [17]:
import gc
import ctypes

In [18]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [19]:
# now we are going to look at a method, that will dig into the garbage collector
# and it will tell us whether that object exist in the garbage collector or not. 
# we will do that again by memory address

def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not Found"

In [20]:
class A:
    def __init__(self_a): #constructor
        self_a.b = B(self_a)
        print("A: self_a: {0}, b: {1}".format(hex(id(self_a)), hex(id(self_a.b))))

In [21]:
class B:
    def __init__(self_b, a):
        self_b.a = a
        print("B: self_b: {0}, a: {1}".format(hex(id(self_b)), hex(id(self_b.a))))

In [22]:
# let's disable the garbage collector
gc.disable()

In [23]:
my_var = A()

B: self_b: 0x1fccc4b3e08, a: 0x1fccc4b3dc8
A: self_a: 0x1fccc4b3dc8, b: 0x1fccc4b3e08


In [24]:
hex(id(my_var))

'0x1fccc4b3dc8'

In [26]:
# another way we can do this is
print(hex(id(my_var.b)))
print(hex(id(my_var.b.a)))
print(hex(id(my_var)))

0x1fccc4b3e08
0x1fccc4b3dc8
0x1fccc4b3dc8


In [27]:
a_id = id(my_var)
b_id = id(my_var.b)

In [28]:
ref_count(a_id)

2

In [29]:
ref_count(b_id)

1

In [30]:
object_by_id(a_id)

'Object exists'

Object Exists in the GC, and it has not cleared (since we disabled it)

In [31]:
object_by_id(b_id)

'Object exists'

In [32]:
# let's destroy my_var
my_var = None

Now, since we destroyed my_var. There sould be no references for the objects A and B, but since it is a cyclic reference we may still have it.

In [33]:
ref_count(a_id)

1

The reference for object A, has reduced from 2 to 1, now only object B is referring to object A, and we will see that object B will have one reference from object A.
Since both are having one references, Python Memory Manager will not kick in and keep it in memory.
Hence leading to Memory Leakage.

In [34]:
ref_count(b_id)

1

In [35]:
object_by_id(a_id)

'Object exists'

In [36]:
object_by_id(b_id)

'Object exists'

In [37]:
# let's call gc manually
gc.collect() # the number of unreachable objects"

664

In [38]:
object_by_id(a_id),object_by_id(b_id)

('Not Found', 'Not Found')

Now check the refences for Object A and B

In [41]:
ref_count(a_id), ref_count(b_id)

(0, 0)

We may also get some garbage number, which means that the memory location is now being used by some other program.

## Quiz

1) **Which one is wrong?**  
a. Parameter is a variable in the decleration of function.  
b. Argument is the actual value of this variable that gets passed to function.  
c. Both are correct.

**c. Both are correct**

2) **What is Garbage Collector**

3) **Say a=10, type(a) gives 'int'. What is int?**
a. The type of a  
b. The type of the object a is pointing to.  
c. The type of the object a is referencing.
d. The type of the object stored at the location where a is pointing.  

**a,b and d**

4) **my_list = [1,2,3], then my_list.append(4). Has the address of my_list changes in these operations?**

**NO**

In [49]:
my_list = [1,2,3]
id(my_list)

2185269782792

In [50]:
my_list.append(4)
id(my_list)

2185269782792

4) **my_list = [1,2,3], then my_list = my_list + [4]. Has the address of my_list changes in these operations?**

**YES**

In [52]:
my_list = [1,2,3]
id(my_list)

2185270945608

In [54]:
my_list = my_list + [4]
id(my_list)

2185269816200

5) **a= 4 , b= 6, Can a nad b be swapped without using 3rd variable?**

**YES**

In [55]:
a,b = 4,6
print(a,b)
b,a = a,b
print(a,b)

4 6
6 4


# Dynamic vs Static Typing

In [42]:
a = "hello"

In [43]:
type(a)

str

In [44]:
a = 10
type(a)

int

In [45]:
a = lambda x:x**2
type(a)

function

In [56]:
a = 11 + 12j
type(a)

complex

In [57]:
a = 10
hex(id(a))

'0x7ffd39d7a2b0'

In [58]:
type(a)

int

# Variable Re-Assignment

In [59]:
print(hex(id(a)))
a = 15
print(hex(id(a)))
a = a + 1
print(hex(id(a)))

0x7ffd39d7a2b0
0x7ffd39d7a350
0x7ffd39d7a370


Here the integer 10 and 15 are not removed from the memory, just the reference pointer of a is moving away. Python **NEVER DELETS THE VALUE OF ANY INT**.

In [70]:
# now this shouldn't surprise you
a = 10
b = 10
print(hex(id(a)))
print(hex(id(b)))
print(a == b)
print(a is b)

0x7ffd39d7a2b0
0x7ffd39d7a2b0
True
True


Integers -5 to 256 are already stored and integer interning occurs, so int from -5 to 256 assigned to any number of variables will always point to same location.

In [71]:
a = [1, 2, 3, 10, 257]
b = [1, 2, 3, 10, 257]
print(hex(id(a)))
print(hex(id(b)))
print(a == b)
print(a is b)

0x1fccc54f8c8
0x1fccc554248
True
False


But not true when done using list

In [72]:
a[3] is b[3], a[4] is b[4]

(True, False)

In [73]:
a = 10
b = 5
c = 15
print(c is a + b)

True


In [74]:
id(15), id(c), id(a+b)

(140725573886800, 140725573886800, 140725573886800)

In [75]:
a = 10
b = 51
c = 61
print(c is a + b)

True


In [76]:
a = 10
b = 5
c = a * b
print(c is a * b)

True


In [81]:
a = 10
b = 5
c = 20
print (c is a + b*2)

True


# Object Mutability

Lay Man Explanations and Examples:

**Immutability:** When an object is not capable of changing the values inside it, than is called muttable object. Like a drawing/number written be a **pen** can be changed.(in normal conditions).

**Mutability:** When an object is capable of changing the values inside it directly, just like a pencil drawing which can be erased and changed

**Changing the data *inside* the object is called modifying the internal state of the object**

**Immutable:**  
* Numbers (float, int, booleans, etc.)  
* Strings  
* Tuple
* Frozen Sets
* User-Defined Class(can be made)

**Mutable**  
* Lists
* Sets
* Dictionaries
* User-Defined Class

### Caution!

t = (1,2,3)  
Tuples are *immutable*: elements **cannot** be deleted, inserted or replaced. In this case, both the container (tuple), and all its elements (ints) are **immutable**

But consider this  
a = [1,2]
b = [3,4]  
Lists are **mutable**: elements **can** be deleted, inserted or replaced

t = (a,b) # t = ([1,2], [3,4])

a.append(3)
b.append(5)

Now t stills points to lists which are updated, since appending to lists doesn't change their address.

t = ([1,2,3], [3,4,5])

In [84]:
my_list = [1, 2, 3]
print(type(my_list))
id(my_list)

<class 'list'>


2185271560200

In [86]:
my_list.append(4)
id(my_list) # address hasn't changed

2185271560200

In [87]:
# another method of putting elements to a list

my_list_1 = [1, 2, 3]
id(my_list_1) # not going to be same as above

2185271617288

In [88]:
my_list_1 = my_list_1 + [4] # concatenation
id(my_list_1)

2185269551368

In [89]:
my_dict = dict(key1=1, key2 = 'a')
my_dict

{'key1': 1, 'key2': 'a'}

In [90]:
id(my_dict)

2185271613144

In [91]:
my_dict['key3'] = 'tsai'
my_dict

{'key1': 1, 'key2': 'a', 'key3': 'tsai'}

In [92]:
id(my_dict)

2185271613144

In [107]:
t = 1,2,3 #or (1,2,3). Both are tuples
t

(1, 2, 3)

In [108]:
id(t)

2185271662856

In [109]:
m = 1
id(m),id(t[0])

(140725573886352, 140725573886352)

In [110]:
t = ([1, 2], [3, 4])
id(t)

2185269790024

In [111]:
t[0], id(t[0])

([1, 2], 2185271755592)

In [112]:
t[0].append(3)
t

([1, 2, 3], [3, 4])

In [113]:
t[0], id(t[0])

([1, 2, 3], 2185271755592)

In [114]:
id(t)

2185269790024

Address of tuple will **NEVER CHANGE**, since it's immutable 

In [117]:
t = ([1, 2], [3, 4])
t[0] = [1,5]

TypeError: 'tuple' object does not support item assignment

In [118]:
t = ([1, 2], [3, 4])
t[0] = t[0] + [5]

TypeError: 'tuple' object does not support item assignment

# Function Arguments and Mutability

In Python, Strings (str) are **Immutable** objects.

Once a string has been created, the content of the objects can never be changed.

The only way to modify the "value" of a variable is to re-assign is to another object

In [119]:
my_var = 'hello'
my_var

'hello'

In [121]:
my_var = 'bonjour' #Re-Assigning my_var to another object.
my_var

'bonjour'

In [126]:
def process(s):
    print(f'Initial s mem-add = {id(s)}')
    s = s + ' world' # concatenating
    print(f'Final s mem-add = {id(s)}')

In [127]:
my_var = 'hello'
print(f'my_var mem-add = {id(my_var)}')

my_var mem-add = 2185270781552


In [132]:
process(my_var)

Initial s mem-add = 2185270781552
Final s mem-add = 2185271911216


In [133]:
print(f'my_var mem-add = {id(my_var)}')
print(my_var) # immutable!

my_var mem-add = 2185270781552
hello


In [134]:
def modify_list(lst):
    print(f'Initial lst mem-add = {id(lst)}')
    lst.append(100)
    print(f'Final lst mem-add = {id(lst)}')

In [136]:
my_list = [1, 2, 3]
print(my_list)
print(f'my_list mem-add = {id(my_list)}')
print('******* STARTING MODIFING *******')
modify_list(my_list)
print('******* ENDING MODIFING *******')
print(my_list)
print(f'my_list mem-add = {id(my_list)}')

[1, 2, 3]
my_list mem-add = 2185271933832
******* STARTING MODIFING *******
Initial lst mem-add = 2185271933832
Final lst mem-add = 2185271933832
******* ENDING MODIFING *******
[1, 2, 3, 100]
my_list mem-add = 2185271933832


In [137]:
def modify_tuple(t):
    print(f'Initial t mem-add = {id(t)}')
    t[0].append(100) # assume first element of the tuple is a list
    print(f'Final t mem-add = {id(t)}')

In [138]:
my_tuple = ([1, 2], 'a')
print(my_tuple)
print(f'my_tuple mem-add = {id(my_tuple)}')
print('******* STARTING MODIFING *******')
modify_tuple(my_tuple)
print('******* ENDING MODIFING *******')
print(my_tuple)
print(f'my_tuple mem-add = {id(my_tuple)}')

([1, 2], 'a')
my_tuple mem-add = 2185270860680
******* STARTING MODIFING *******
Initial t mem-add = 2185270860680
Final t mem-add = 2185270860680
******* ENDING MODIFING *******
([1, 2, 100], 'a')
my_tuple mem-add = 2185270860680


# Shared Reference and Mutability

* The term shared reference is the concept of two variables referencing the same object in the memory (i.e. having the same memory address)
 * a = 10, then when we write b = a, what we are actually saying is, set the memory reference of b equal to the memory reference of a. 
 * It is not copying "values", it is copying memory references. 
 
* Same thing with a function too. When you pass an argument to a function, what you are actually passing is the memory reference. 

* Shared references happen all the time in our code, and we need to be careful with whether the reference address belongs to a mutable or immutable object. 

* In some cases, even if you declare two separate variables separately, Python's memory manager might decide to automatically re-use the memory references! e.g.:
 * a = 10 and b = 10; a and b will have same memory addresses
 * s1 = "hello" and s2 = "hello"; s1 and s2 will have same memory addresses
 
* How is that last thing safe?
 * Well YES, that happens only for immutable objects, so you can't change what's stored on those addresses anyways. Python is doing this for optimizations (we'll cover this when we cover python optimizations).
 
* **With Mutable objects, Python Memory Manager will never create shared references.** (Exception: In Immutable tuple will also not create shared reference)
 * a = [1, 2, 3] and b = [1, 2, 3] will be stored at different memory addresses. 

In [139]:
a = "hello"
b = a # manually created shared reference

print(id(a))
print(id(b))

2185270781552
2185270781552


In [140]:
a is b

True

In [141]:
a = "hello"
b = "hello" # automatically created shared reference

print(id(a))
print(id(b))

2185270781552
2185270781552


In [142]:
# this is safe, because there is no way to "change" b.. 
b = "hello world"
print(id(b)) # this is not the same b "tum badal gae!"

2185271980976


In [147]:
a = [1, 2, 3]
b = a # shared reference
print(id(a))
print(id(b))

2185272000200
2185272000200


In [148]:
b.append('4')
print(id(a))
print(id(b))
print(f'a: {a}')
print(f'b: {b}')

2185272000200
2185272000200
a: [1, 2, 3, '4']
b: [1, 2, 3, '4']


We never appended anything to list **'a'**. But since we are pointing **b** to same list as **a**, changing anything in **b** will automatically change the object(list) in that memory location, and will effect all the variables pointing to that object(list). Hence also changing **a**

# Variable Equality

**Memory Address** ***is*** *identity* operator --> var_1 is var_2

**Object State** ***==*** --> var_1 == var_2

The **None** object can be assigned to variables to indicate that they are not set (in the way we would expect them to be, i.e. an "empty" value or null pointer)

But the None object is a **real** object that is managed bt the Python Memory Manager.

Furthermore, the memory manager will always used a **shared reference** when assigning a variable to **None**

In [149]:
a = None
b = None
a == b

True

In [157]:
None.__dir__()

['__repr__',
 '__bool__',
 '__new__',
 '__doc__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [152]:
a = [1, 2, 3]
b = [1, 2, 3]
print(id(a))
print(id(b))
print(f'a is b {a is b}')
print(f'a == b {a == b}')

2185272065864
2185272065800
a is b False
a == b True


In [153]:
a = 1, 2, 3
b = 1, 2, 3
print(id(a))
print(id(b))
print(f'a is b {a is b}')
print(f'a == b {a == b}')

2185272016792
2185271767432
a is b False
a == b True


In [154]:
a = 10
b = 10.0
print(id(a))
print(id(b))
print(f'a is b {a is b}')
print(f'a == b {a == b}') # <<<<

140725573886640
2185272001296
a is b False
a == b True


In [155]:
a = 10 + 0j
b = 10.0
print(id(a))
print(id(b))
print(f'a is b {a is b}')
print(f'a == b {a == b}')

2185272000944
2185272000752
a is b False
a == b True


In [158]:
print(id(None)) # has a memory address
print(type(None))

140725573410016
<class 'NoneType'>


In [159]:
a = None
b = None
c = None

In [160]:
a is b
b is None

True

# Everything is an OBJECT!

In [161]:
a = 10
print(type(a))

<class 'int'>


In [162]:
b = int(10) # isn't this how we instantiate an instance for any class
print(type(b))

<class 'int'>


In [163]:
c = int()
c

0

In [164]:
c = int('101', base = 2)
c

5

In [165]:
def square(a):
    return a **2

In [166]:
type(square)

function

In [167]:
f = square
f(3)

9

In [168]:
print(id(square))
print(id(f))
f is square

2185271866856
2185271866856


True

In [169]:
print(square(2))
print(f(2))

4
4


In [170]:
def cube(a):
    return a ** 3

In [171]:
def select_function(fn_id):
    if fn_id == 1:
        return square
    else:
        return cube

In [172]:
f = select_function(1)
f is square

True

In [173]:
f(2)

4

In [174]:
f = select_function(2)
f is cube

True

In [175]:
select_function(2)(4)

64

In [176]:
# a function can be passed to a function as well
def exec_function(fn, n):
    return fn(n)

print(exec_function(square, 2))
print(exec_function(cube, 3))

4
27


# Python Optimizers - Integer Interning

### Important Note:

A lot of what was discussed with memory management, garbage collection and optimization, is usually specific to Python implementation being used.

We are (and will be using) **CPython**, the standard (or reference) Python implementation, which is written in **C**

**Interning:** Resuing objects on-demand.

At startup, CPython **pre-loads**(caches) a global list on integers in the range -[5,256].  
Any time an integer is reference from this range, Python will use the cached version of that object.

Integers in the range -5 to 256 are **singleton** object, i.e. classes that can only be initialized **ONCE**. This is an optimization strategy (since small integers often show up.)

In [177]:
# [-5, 256]

a = 10
b = 10
print(id(a))
print(id(b))

140725573886640
140725573886640


In [178]:
a = -5
b = -5
print(id(a))
print(id(b))

140725573886160
140725573886160


In [179]:
a = 257
b = 257
print(id(a))
print(id(b))

2185272003920
2185272003888


In [180]:
a = 10
b = int(10)
c = int('10')
d = int('1010', 2)
print(id(a))
print(id(b))
print(id(c))
print(id(d))
a is b is c is d

140725573886640
140725573886640
140725573886640
140725573886640


True

# Python Optimizers - String Interning

Some string are also automatically interned - but not all.

As Python code is compiled, identifiers are interned:  
 * variable name
 * fnction name
 * class names etc.

In [181]:
a = 'tsai'
b = 'tsai'
print(id(a))
print(id(a))
print(a is b)

2185271649136
2185271649136
True


In [182]:
a = 'hello world'
b = 'hello world'
print(id(a))
print(id(a))
print(a is b)

2185272223536
2185272223536
False


In [183]:
a = 'hello_world'
b = 'hello_world'
print(id(a))
print(id(a))
print(a is b)

2185272238320
2185272238320
True


In [184]:
a = "_this_is_a_long_string_which_might_get_interned"
b = "_this_is_a_long_string_which_might_get_interned"
print(id(a))
print(id(a))
print(a is b)

2185272235376
2185272235376
True


In [185]:
# force string interning
import sys

In [186]:
a = sys.intern("hello world")
b = sys.intern("hello world")
c = "hello world"
print(id(a),id(b),id(c)) 

2185272263152 2185272263152 2185272263344


In [189]:
print(a == b) # character by character test
print(a is b) # int comparasion, extremely fast

True
True


In [190]:
def compare_using_equals(n):
    a = 'a long string that is not intered' * 200
    b = 'a long string that is not intered' * 200
    for i in range(n):
        if a == b:
            pass

In [191]:
def compare_using_interning(n):
    a = sys.intern('a long string that is not intered' * 200)
    b = sys.intern('a long string that is not intered' * 200)
    for i in range(n):
        if a is b:
            pass

In [192]:
import time

In [193]:
start = time.perf_counter()
compare_using_equals(10000000)
end = time.perf_counter()
print('equality', end-start)

equality 3.275232699990738


In [194]:
start = time.perf_counter()
compare_using_interning(10000000)
end = time.perf_counter()
print('interning', end-start)

interning 0.4021619999984978


# Python Optimizers - Peephole

There is another variety of optimizations that can occur at compile time - **peephole**

Constant Expressions:
 * numeric calculations: 24\*60 Constant expression gets precalculated and stored
 * short sequences < 4096: (1,2)*5 --> (1,2,1,2,1,2,1,2,1,2)
 
Membership Test:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;When membership tests such as: **if e in [1,2,3]:** are encountered, the [1,2,3] is replaced by immutable counterpart (1,2,3) - a tuple.  
Lists are converted to tuples and sets are converted to frozen sets

**Sets** membership is much faster than list or tuple membership as they are like dictionaries (hash)

So instead of **if e in [1,2,3]:** use **if e in {1,2,3}:** LOGICALLY

In [200]:
def my_func():
    a = 24 * 60
    b = (1, 2) * 5
    c = 'abc' * 3
    d = 'ab' * 2050
    e = 'the quick brown fox is awesome' * 500
    f = ['a', 'b'] * 3 # list is not constant

# let's compile this.. this will add constants to code associated with this function.. 
# now let's check that

In [201]:
my_func.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'ab',
 2050,
 'the quick brown fox is awesome',
 500,
 'a',
 'b',
 3)

In [202]:
def my_func():
    a = 24 * 60
    b = (1, 2) * 5
    c = 'abc' * 3
    d = 'ab' * 2000
    e = 'the quick brown fox is awesome' * 500
    f = ['a', 'b'] * 3 # list is not constant

# let's compile this.. this will add constants to code associated with this function.. 
# now let's check that

In [203]:
my_func.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'ababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababababa

In [204]:
def my_func(e):
    if e in [1, 2, 3]:
        pass

In [205]:
my_func.__code__.co_consts

(None, (1, 2, 3))

In [206]:
def my_func(e):
    if e in {1, 2, 3}:
        pass
my_func.__code__.co_consts

(None, frozenset({1, 2, 3}))

In [207]:
import string
import time

In [209]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [214]:
char_list = list(string.ascii_letters)
char_tuple = tuple(string.ascii_letters)
char_set = set(string.ascii_letters)

In [215]:
print(char_list)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [216]:
print(char_tuple)

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z')


In [218]:
print(char_set) # order not guaranteed, just like dictionary

{'h', 'K', 'B', 't', 'j', 'F', 'U', 'X', 'k', 'r', 'J', 'b', 'm', 'd', 'G', 'C', 'l', 'w', 'Z', 's', 'A', 'e', 'I', 'f', 'D', 'v', 'N', 'T', 'Q', 'H', 'z', 'x', 'O', 'S', 'i', 'E', 'R', 'V', 'W', 'Y', 'o', 'c', 'n', 'y', 'L', 'P', 'u', 'g', 'a', 'q', 'p', 'M'}


In [219]:
def membership_test(n, container):
    for i in range(n):
        if 'z' in container: # leveraging polymorphism.. as long as containers support in operator
            pass

In [220]:
start = time.perf_counter()
membership_test(10000000, char_list)
end = time.perf_counter()
print('list: ', end - start)

list:  4.474197199990158


In [221]:
start = time.perf_counter()
membership_test(10000000, char_tuple)
end = time.perf_counter()
print('tuple: ', end - start)

tuple:  4.466291800010367


In [222]:
start = time.perf_counter()
membership_test(10000000, char_set)
end = time.perf_counter()
print('set: ', end - start)

set:  0.40880059999471996
